## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-08-19-05-23 +0000,nypost,New book exposes Jill Biden’s power grab amid ...,https://nypost.com/2025/07/08/us-news/new-book...
1,2025-07-08-18-48-57 +0000,nypost,Trump calls Staff Gen. Milley ‘idiot’ for leav...,https://nypost.com/2025/07/08/us-news/donald-t...
2,2025-07-08-18-48-53 +0000,nyt,How Harvard’s Ties to China Helped Make It a W...,https://www.nytimes.com/2025/07/07/us/harvard-...
3,2025-07-08-18-46-11 +0000,nyt,Agriculture Dept. to Crack Down on Chinese Own...,https://www.nytimes.com/2025/07/08/us/politics...
4,2025-07-08-18-38-59 +0000,nyt,Deadly Flash Floods Sweep Away Nepal-China Bridge,https://www.nytimes.com/2025/07/08/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,21
200,tariffs,6
0,new,6
104,texas,5
260,fire,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
27,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,44
50,2025-07-07-23-24-42 +0000,nypost,EPA chief Lee Zeldin unveils Trump admin plan ...,https://nypost.com/2025/07/07/us-news/epa-chie...,43
30,2025-07-08-13-42-22 +0000,nypost,Trump admin launches National Farm Security Ac...,https://nypost.com/2025/07/08/us-news/agricult...,42
14,2025-07-08-16-43-22 +0000,nypost,Trump rips Putin during cabinet meeting for ta...,https://nypost.com/2025/07/08/us-news/trump-ri...,42
49,2025-07-07-23-26-33 +0000,nypost,Netanyahu nominates Trump for Nobel Peace Priz...,https://nypost.com/2025/07/07/us-news/trump-ho...,41


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
27,44,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...
53,26,2025-07-07-22-06-53 +0000,nypost,White House urges Dems to ‘tone down’ anti-ICE...,https://nypost.com/2025/07/07/us-news/white-ho...
50,22,2025-07-07-23-24-42 +0000,nypost,EPA chief Lee Zeldin unveils Trump admin plan ...,https://nypost.com/2025/07/07/us-news/epa-chie...
25,21,2025-07-08-14-47-33 +0000,nypost,AI-powered Marco Rubio impersonator makes call...,https://nypost.com/2025/07/08/us-news/ai-power...
14,21,2025-07-08-16-43-22 +0000,nypost,Trump rips Putin during cabinet meeting for ta...,https://nypost.com/2025/07/08/us-news/trump-ri...
45,20,2025-07-08-03-07-20 +0000,nypost,Gov. Ron DeSantis warns Elon Musk about runnin...,https://nypost.com/2025/07/07/us-news/desantis...
59,18,2025-07-07-21-12-29 +0000,nypost,Sen. Ted Cruz cuts short European vacation due...,https://nypost.com/2025/07/07/us-news/ted-cruz...
43,17,2025-07-08-04-00-11 +0000,nyt,Trump and Netanyahu Meet Amid Gaza Cease-Fire ...,https://www.nytimes.com/2025/07/07/us/politics...
58,16,2025-07-07-21-23-28 +0000,nypost,Florida business group woos NYC CEOs worried a...,https://nypost.com/2025/07/07/us-news/florida-...
40,14,2025-07-08-04-34-03 +0000,nypost,Kamala Harris campaign staffers warned not to ...,https://nypost.com/2025/07/08/us-news/kamala-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
